In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import email
import warnings 
warnings.filterwarnings('ignore')
import parse
import re
import math

In [5]:
df_message=pd.read_json('/Users/kshitijmittal/Documents/UChicago Acad/Hedwig_AI/Hedwig/02_Data_Ingestion/messages_v2.json')
df_message.head()

,time,subject,thread,sender,recipient,cc,bcc,message
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,0,0,[1],[],[],Here is our forecast
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,1,0,[2],[],[],Traveling to have a business meeting takes the...
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,2,0,[3],[],[],test successful. way to go!!!
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",,0,0,[4],[],[],"Randy, Can you send me a schedule of the salar..."
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.


In [11]:
df_message.shape

(517401, 8)

In [13]:
pd.options.display.max_colwidth = None
df_message[df_message.thread==4].head(30)

,time,subject,thread,sender,recipient,cc,bcc,message
7,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",Re: PRC review - phone calls,4,0,[8],[],[],any morning between 10 and 11:30
1213,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",Re: PRC review - phone calls,4,0,[8],[],[],any morning between 10 and 11:30
3000,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",Re: PRC review - phone calls,4,0,[8],[],[],any morning between 10 and 11:30


In [14]:
df_message_v2=df_message.drop_duplicates(subset=['message'], keep='first')

In [17]:
df_message_v2.shape
df_message_v2.head(30)

,time,subject,thread,sender,recipient,cc,bcc,message
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,0,0,[1],[],[],Here is our forecast
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,1,0,[2],[],[],"Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary. As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. My suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time."
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,2,0,[3],[],[],test successful. way to go!!!
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",,0,0,[4],[],[],"Randy, Can you send me a schedule of the salary and level of everyone in the scheduling group. Plus your thoughts on any changes that need to be made. (Patti S for example) Phillip"
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.
5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],"Greg, How about either next Tuesday or Thursday? Phillip"
6,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",,0,0,"[6, 7]",[],[],Please cc the following distribution list with updates: Phillip Allen (pallen@enron.com) Mike Grigsby (mike.grigsby@enron.com) Keith Holst (kholst@enron.com) Monique Sanchez Frank Ermis John Lavorato Thank you for your help Phillip Allen
7,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",Re: PRC review - phone calls,4,0,[8],[],[],any morning between 10 and 11:30
8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think these are required by the ISP 2. static IP address IP: 64.216.90.105 Sub: 255.255.255.248 gate: 64.216.90.110 DNS: 151.164.1.8 3. Company: 0413 RC: 105891
9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",FW: fixed forward or other Collar floor gas price terms,6,0,[10],[],[],"---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/16/2000 01:42 PM --------------------------- ""Buckner, Buck"" <buck.buckner@honeywell.com> on 10/12/2000 01:12:21 PM To: ""'Pallen@Enron.com'"" <Pallen@Enron.com> cc: Subject: FW: fixed forward or other Collar floor gas price terms Phillip, > As discussed during our phone conversation, In a Parallon 75 microturbine > power generation deal for a national accounts customer, I am developing a > proposal to sell power to customer at fixed or collar/floor price. To do > so I need a corresponding term gas price for same. Microturbine is an > onsite generation product developed by Honeywell to generate electricity > on customer site (degen). using natural gas. In doing so, I need your > best fixed price forward gas price deal for 1, 3, 5, 7 and 10 years for > annual/seasonal supply to microturbines to generate fixed kWh for > customer. We have the opportunity to sell customer kWh 's using > microturbine or sell them turbines themselves. kWh deal must have limited/ > no risk forward gas price to make deal work. Therein comes Sempra energy > gas trading, truly you. > > We are proposing installing 180 - 240 units across a large number of > stores (60-100) in San Diego. > Store number varies because of installation hurdles face at small percent. > > For 6-8 hours a day Microturbine run time: > Gas requirement for 180 microturbines 227 - 302 MMcf per year > Gas requirement for 240 microturbines 302 - 403 MMcf per year > > Gas will likely be consumed from May through September, during peak > electric period. > Gas price required: Burnertip price behind (LDC) San Diego Gas & E